In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from __future__ import print_function, division
from builtins import range

In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [5]:
from nltk.corpus import stopwords


In [6]:
ignore_words = set(stopwords.words('english')).union(set(stopwords.words('indonesian')))
print(len(ignore_words))

936


In [7]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_cat=pd.read_json('categories.json')

In [8]:
df_train.title.head()



0                 nyx sex bomb pallete natural palette
1    etude house precious mineral any cushion pearl...
2                             milani rose powder blush
3                  etude house baby sweet sugar powder
4         bedak revlon color stay aqua mineral make up
Name: title, dtype: object

In [9]:
df_train['Group']=df_train.image_path.map(lambda x: x[:7])
df_test['Group']=df_test.image_path.map(lambda x: x[:7])

In [10]:
train,test=train_test_split(df_train,random_state=2019,stratify=df_train.Category,test_size=0.2)

In [11]:
np.random.seed(2019)

In [12]:
groups=['beauty_','mobile_','fashion']

In [13]:
# some linear models
from sklearn.linear_model import LogisticRegression, BayesianRidge

# SCM for classification
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

import warnings
warnings.filterwarnings('ignore')
np.random.seed(2019)


In [14]:
from sklearn.metrics import accuracy_score

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [16]:
# let's build pipeline for parameter search in n_grams and and various models
lr=LogisticRegression()
nb=MultinomialNB()
svc=SVC()
# let's check first three and see which one is best
gnb=GaussianNB()
bnb=BernoulliNB()
br=BayesianRidge()
rf=RandomForestClassifier(n_estimators=150)

In [17]:
# Best cv scores for parameter for lr.


In [18]:
def submission(name,dic):
    df_sub=df_test.copy()
    for group in dic.keys():
        df_sub.loc[df_sub.Group==group,'Category']=dic[group]
    df_sub.loc[:,'Category']=df_sub.Category.astype('int')
    df_sub[['itemid','Category']].to_csv(name,index=False)

## Let's do ensembling now! 
## So we have results from:
## 1) CountVectorization with logistic regression : lr ending
## 2) CountVectorization with Naive Bayes:  nb ending
## 3) Image categorization:  im ending
## 4) word2vec predictions: w2v ending


## 1 and 2 might be highly correlated with each other. I might remove Naive Bayes. But we also can try with and without and observe results.


In [19]:
df_lr={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_test_proba_nlp_val_data2.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_nlp_all_data2.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_lr[groups[i]]=val_test

In [20]:
# own corpus in word2vec unlike original of this code
df_w2v={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_test_proba_w2v_own_val_data.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_w2v_own_all_data.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_w2v[groups[i]]=val_test

In [21]:
df_nb={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_val_proba_nlp_nb.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_nlp_nb.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_nb[groups[i]]=val_test

In [22]:
# We have previously ordered outputof image predictions so we can just upload all of them
df_im={}
for group in groups:
    if group=='mobile_':
        val_test=[]
        df1=pd.read_csv("val_mobile__v2.csv") 
        df2=pd.read_csv("test_all_mobile__v2.csv") # 97% of mobile data
        val_test.append(df1)
        val_test.append(df2)
        df_im[group]=val_test # Overite df_im but I use df_im in the rest of the code
    elif group=='fashion':
        val_test=[]
        df1=pd.read_csv('val_fashion_v2.csv') 
        df2=pd.read_csv('test_all_fashion_v2.csv') # trained on 100% of data
        val_test.append(df1)
        val_test.append(df2)
        df_im[group]=val_test # Overite df_im but I use df_im in the rest of the code
    else:
        val_test=[]
        df1=pd.read_csv('val_beauty__v2.csv') 
        df2=pd.read_csv('test_beauty__v2.csv') # trained on 80% of data
        val_test.append(df1)
        val_test.append(df2)
        df_im[group]=val_test # Overite df_im but I use df_im in the rest of the code


## We need to check impact of fashion image classification improvement on predictions.


# finally, we can start ensembling. Let's first check their relative error in internal test set. Also, correlations btw them

In [72]:
results=[df_lr,df_nb,df_w2v,df_im]

In [73]:
# Let's define columns for each category
columns={}
columns['beauty_']=[str(i) for i in range(17)]
columns['mobile_']=[str(i) for i in range(31,58)]
columns['fashion']=[str(i) for i in range(17,31)]

In [74]:
# df_im is actually df_im2 we overwrite it in the above code
models={'lr':df_lr,'nb':df_nb,'w2v':df_w2v,'im':df_im}

In [75]:
for name, model in models.items():
    if name=='im':
        pred='final_pred'
    else:
        pred='pred'
    print('Accuracy score for model: ', name)
    for group in groups:
        print(group,accuracy_score(test[test.Group==group].Category,model[group][0][pred]))

Accuracy score for model:  lr
beauty_ 0.789800582724
mobile_ 0.830687665679
fashion 0.652056166223
Accuracy score for model:  nb
beauty_ 0.764188635134
mobile_ 0.778949009824
fashion 0.592362486061
Accuracy score for model:  w2v
beauty_ 0.752324790202
mobile_ 0.776360517698
fashion 0.601351812658
Accuracy score for model:  im
beauty_ 0.69837918942
mobile_ 0.663152970529
fashion 0.525727680299


Previous results of word2vec based on combination of english and indonesian corpuses
Our own word2vec seems to be much better in all of groups

Accuracy score for model:  w2v
beauty_ 0.744054992411
mobile_ 0.766505535631
fashion 0.579777428825


In [27]:
df_all={}
for group in groups:
    val_test=[pd.DataFrame(),pd.DataFrame()]
    df_all[group]=val_test

In [28]:
len(models.keys())

4

In [29]:
#  Let's put all of the predictions together for ensembling
# I am still not sure about best stragery for ensembling
df_all_val={}
df_all_test={}
for group in groups:
    df_all_val[group]=models['lr'][group][0][columns[group]].copy()
    df_all_test[group]=models['lr'][group][1][columns[group]].copy()
    for name, model in models.items():
        if name!='lr':
            df1=models[name][group][0][columns[group]].copy()
            df_all_val[group]=df_all_val[group].join(df1,rsuffix=name)
            
            df2=models[name][group][1][columns[group]].copy()
            df_all_test[group]=df_all_test[group].join(df2,rsuffix=name)
        
# Now we put everything together.

In [30]:
# Sanity check
for group in groups:
    print("shape of df_all_val and test", group)
    print(df_all_val[group].shape, test[test.Group==group].shape)

shape of df_all_val and test beauty_
(57317, 68) (57317, 5)
shape of df_all_val and test mobile_
(32065, 108) (32065, 5)
shape of df_all_val and test fashion
(43941, 56) (43941, 5)


In [31]:
for group in groups:
    print("shape of df_all_test and df_test", group)
    print(df_all_test[group].shape, df_test[df_test.Group==group].shape)

shape of df_all_test and df_test beauty_
(76545, 68) (76545, 4)
shape of df_all_test and df_test mobile_
(40417, 108) (40417, 4)
shape of df_all_test and df_test fashion
(55440, 56) (55440, 4)


In [32]:
from sklearn.neighbors import KNeighborsClassifier

In [33]:
lr=LogisticRegression()
svm=SVC(kernel='linear')
rf=RandomForestClassifier(n_estimators=200)
knn=KNeighborsClassifier(n_neighbors=50)

In [34]:
from sklearn.model_selection import StratifiedKFold

In [35]:
pipe_en2={}
pipe_en2['fashion']=RandomForestClassifier(n_estimators=280)
pipe_en2['beauty_']=RandomForestClassifier(n_estimators=280)
pipe_en2['mobile_']=RandomForestClassifier(n_estimators=220)

In [36]:
pipe_en2['fashion']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=280, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
groups

['beauty_', 'mobile_', 'fashion']

In [38]:
# I am not sure but we might be overfitting. Let's try our previous approach


def eval_model():
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    predictions2={}
    for group in groups:
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2


In [39]:
pred=eval_model()

Results for group beauty_
Fold: 0
Acuracy of fold: 0.790496948561
Fold: 1
Acuracy of fold: 0.789271696468
Fold: 2
Acuracy of fold: 0.799703393527
Fold: 3
Acuracy of fold: 0.794189495725
Fold: 4
Acuracy of fold: 0.789560967094
Acuracy of overall: 0.792644416142
Results for group mobile_
Fold: 0
Acuracy of fold: 0.831983805668
Fold: 1
Acuracy of fold: 0.83232039894
Fold: 2
Acuracy of fold: 0.829591518553
Fold: 3
Acuracy of fold: 0.831539541413
Fold: 4
Acuracy of fold: 0.835806905171
Acuracy of overall: 0.832246998285
Results for group fashion
Fold: 0
Acuracy of fold: 0.677434030937
Fold: 1
Acuracy of fold: 0.670458423388
Fold: 2
Acuracy of fold: 0.671899886234
Fold: 3
Acuracy of fold: 0.666211448731
Fold: 4
Acuracy of fold: 0.658808791709
Acuracy of overall: 0.668965203341


In [40]:
predictions2={}
for group in groups:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [41]:
for group in groups:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(76545, 4) (76545,)
(40417, 4) (40417,)
(55440, 4) (55440,)


In [42]:
for group in groups:
    print(group)
    df_test.loc[df_test.Group==group,'Category']=predictions2[group]

beauty_
mobile_
fashion


In [43]:
df_test['Category']=df_test.Category.astype('int')

In [44]:
df_test[['itemid','Category']].to_csv('ensemble_6_v1.csv',index=False)

In [45]:
# Let's try to do more folds
def eval_model10():
    folds=StratifiedKFold(n_splits=10,shuffle=True,random_state=2019)
    predictions2={}
    for group in groups:
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2

In [46]:
pred=eval_model10()

Results for group beauty_
Fold: 0
Acuracy of fold: 0.787208086441
Fold: 1
Acuracy of fold: 0.793097437685
Fold: 2
Acuracy of fold: 0.794107391911
Fold: 3
Acuracy of fold: 0.787656903766
Fold: 4
Acuracy of fold: 0.794174079888
Fold: 5
Acuracy of fold: 0.802512212142
Fold: 6
Acuracy of fold: 0.792248603352
Fold: 7
Acuracy of fold: 0.796752226296
Fold: 8
Acuracy of fold: 0.787460705554
Fold: 9
Acuracy of fold: 0.794374563242
Acuracy of overall: 0.792958459096
Results for group mobile_
Fold: 0
Acuracy of fold: 0.8328154133
Fold: 1
Acuracy of fold: 0.830171073095
Fold: 2
Acuracy of fold: 0.8304293715
Fold: 3
Acuracy of fold: 0.828455790785
Fold: 4
Acuracy of fold: 0.833749220212
Fold: 5
Acuracy of fold: 0.828027465668
Fold: 6
Acuracy of fold: 0.829481574016
Fold: 7
Acuracy of fold: 0.8321875
Fold: 8
Acuracy of fold: 0.83901219131
Fold: 9
Acuracy of fold: 0.834428794992
Acuracy of overall: 0.831872758459
Results for group fashion
Fold: 0
Acuracy of fold: 0.683037744429
Fold: 1
Acuracy of fol

In [52]:
for group in groups:
    print(df_all_val[group].shape)

(57317, 68)
(32065, 108)
(43941, 56)


In [77]:
# I am not sure but we might be overfitting. Let's try our previous approach


def eval_model():
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    predictions2={}
    for group in groups:
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2


In [90]:
fold={'fashion':10,'beauty_':5,'mobile_':5}
def eval_model_2():
    predictions2={}
    for group in groups:
        folds=StratifiedKFold(n_splits=fold[group],shuffle=True,random_state=2019)
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2

In [80]:
pipe_en2={}
pipe_en2['fashion']=RandomForestClassifier(n_estimators=280,max_features=8,max_depth=20, min_samples_split=5)
pipe_en2['beauty_']=RandomForestClassifier(n_estimators=280,max_features=20,min_samples_leaf=2,max_depth=20, min_samples_split=10)
pipe_en2['mobile_']=RandomForestClassifier(n_estimators=220,max_features=20)

In [81]:
pred=eval_model()

Results for group beauty_
Fold: 0
Acuracy of fold: 0.793897122929
Fold: 1
Acuracy of fold: 0.792411687745
Fold: 2
Acuracy of fold: 0.800750239902
Fold: 3
Acuracy of fold: 0.795498167859
Fold: 4
Acuracy of fold: 0.790171947281
Acuracy of overall: 0.794546120697
Results for group mobile_
Fold: 0
Acuracy of fold: 0.83494238555
Fold: 1
Acuracy of fold: 0.833255415303
Fold: 2
Acuracy of fold: 0.833645151232
Fold: 3
Acuracy of fold: 0.832007487131
Fold: 4
Acuracy of fold: 0.835338228402
Acuracy of overall: 0.833837517542
Results for group fashion
Fold: 0
Acuracy of fold: 0.675955414013
Fold: 1
Acuracy of fold: 0.669662154476
Fold: 2
Acuracy of fold: 0.672241183163
Fold: 3
Acuracy of fold: 0.667463298054
Fold: 4
Acuracy of fold: 0.659719849675
Acuracy of overall: 0.669010718919


In [83]:
predictions2={}
for group in groups:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [84]:
for group in groups:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(76545, 5) (76545,)
(40417, 5) (40417,)
(55440, 5) (55440,)


In [85]:
for group in groups:
    print(group)
    df_test.loc[df_test.Group==group,'Category']=predictions2[group]

beauty_
mobile_
fashion


In [86]:
df_test['Category']=df_test.Category.astype('int')

In [87]:
#LB:0.76865
df_test[['itemid','Category']].to_csv('ensemble_7.csv',index=False)

In [91]:
pred=eval_model_2()

Results for group beauty_
Fold: 0
Acuracy of fold: 0.792589363557
Fold: 1
Acuracy of fold: 0.790492804187
Fold: 2
Acuracy of fold: 0.799965105121
Fold: 3
Acuracy of fold: 0.794887454196
Fold: 4
Acuracy of fold: 0.790782927468
Acuracy of overall: 0.793743566481
Results for group mobile_
Fold: 0
Acuracy of fold: 0.83245094986
Fold: 1
Acuracy of fold: 0.833722923484
Fold: 2
Acuracy of fold: 0.833021515435
Fold: 3
Acuracy of fold: 0.831695523319
Fold: 4
Acuracy of fold: 0.836119356351
Acuracy of overall: 0.833400904413
Results for group fashion
Fold: 0
Acuracy of fold: 0.68758526603
Fold: 1
Acuracy of fold: 0.668030923147
Fold: 2
Acuracy of fold: 0.667121418827
Fold: 3
Acuracy of fold: 0.678189674778
Fold: 4
Acuracy of fold: 0.674630261661
Fold: 5
Acuracy of fold: 0.670004551661
Fold: 6
Acuracy of fold: 0.681538811746
Fold: 7
Acuracy of fold: 0.658925318761
Fold: 8
Acuracy of fold: 0.649658314351
Fold: 9
Acuracy of fold: 0.669630642955
Acuracy of overall: 0.670535490772


In [92]:
predictions2={}
for group in groups:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [93]:
for group in groups:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(76545, 5) (76545,)
(40417, 5) (40417,)
(55440, 5) (55440,)


In [94]:
for group in groups:
    print(group)
    df_test.loc[df_test.Group==group,'Category']=predictions2[group]

beauty_
mobile_
fashion


In [95]:
df_test['Category']=df_test.Category.astype('int')

In [105]:
df_test.head()

,itemid,title,image_path,Group,Category
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg,beauty_,5
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg,beauty_,5
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg,beauty_,5
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg,beauty_,5
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg,beauty_,5


In [103]:
df_test[df_test.Group=='fashion'].index

Int64Index([ 76545,  76546,  76547,  76548,  76549,  76550,  76551,  76552,
             76553,  76554,
            ...
            131975, 131976, 131977, 131978, 131979, 131980, 131981, 131982,
            131983, 131984],
           dtype='int64', length=55440)

In [96]:
# We will change only fashion from previous submission
df=pd.read_csv('ensemble_7.csv')

In [106]:
df.loc[df_test[df_test.Group=='fashion'].index,'Category']=df_test[df_test.Group=='fashion'].Category

In [108]:
#LB: 0.76788
df_test[['itemid','Category']].to_csv('ensemble_7_v2.csv',index=False)